This notebook contains the commands to export selected visualizations as HTML.

Just run this notebook to save the visualizations as interactive HTML maps. Use the [visualize](./visualize.ipynb) notebook if you want to learn how to create dataframes and visualizations from PostGIS data.

In [1]:
import sys
from ipygis import get_connection_url, QueryResult, get_map, generate_map
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from kepler_h3_config import config  # we may use our own custom visualization config

In [2]:
sql_url = get_connection_url(dbname='geoviz-parquet')
engine = create_engine(sql_url)
session = sessionmaker(bind=engine)()

sys.path.insert(0, '..')
from models import PopulationHex, OSMPoint, GooglePoint
from osm_tags import tag_filter

In [3]:
places_query = session.query(OSMPoint).filter(tag_filter)
population_query = session.query(PopulationHex)
google_query = session.query(GooglePoint)

You may visualize all points in any database query by simply

In [4]:
get_map(places_query)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'data_1', 'type': '…

Datasets
--------

The next phase will take a while. Here we actually create all the dataframes in memory (using H3 hex resolution 8), from the whole database.

The **places** dataset simply sums the *number of amenity POIs* in each H3 hex. The larger areas you have imported, the longer this query will take:

In [5]:
places = QueryResult.create(places_query, resolution=8, name='places')

The **google_pois** sums the *number of google POIs*, only in Helsinki:

In [6]:
google_pois = QueryResult.create(google_query, resolution=8, name='google_pois')

The **population_around_pois** dataset is the *average population within 400 meters of each poi in the hex*:

In [8]:
population_around_pois = QueryResult.create(places_query, resolution=8, name='population_around_pois', plot='mean', column='populations_200.total')

AttributeError: 'NoneType' object has no attribute 'values'

The **morning popularity** dataset is the *total number of morning visitors* in all Google points within the hex:

In [9]:
morning = QueryResult.create(google_query, resolution=8, name='morning', plot='sum', column='popularity.morning')

AttributeError: 'NoneType' object has no attribute 'items'

The **population** dataset contains the *approximate population* in the H3 hex. With 1.3 million hexes, this is slow:

In [10]:
population = QueryResult.create(population_query, resolution=8, name='population', plot='sum', column='total')

ValueError: not enough values to unpack (expected 2, got 1)

Visualization
-------------

Once the QueryResults exist, calculating the sum map from dataframes should be relatively fast. Layer weights in the sum may be customized here if you want to emphasize or leave out some datasets.

(Note that access layer has *negative* weight, because smaller walking times indicate better pedestrian access to amenities.)

In [11]:
results = [places, population]
columns = ['size', 'total']
weights = [1, 1]

In [7]:
result_map = generate_map(results, 500, config=config, column=columns, weights=weights)
result_map.save_to_html()
result_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to keplergl_map.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hex8_places', 'typ…